In [6]:
import numpy as np
import sys
sys.path.append(r"C:\Users\gapar\Logical-Clifford-Synthesis")
from Algorithms.algorithm_1 import SymplecticOperations
from Algorithms.algorithm_2 import FindAllSympMat
from Algorithms.algorithm_3 import SymplecticCodeSolver
from helper_functions.helperfunctions import SymplecticMatrix
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Initialize the SymplecticOperations class
symplectic_ops = SymplecticOperations()
symp_mat = SymplecticMatrix(4)
symp_code = SymplecticCodeSolver()
solver = FindAllSympMat()

In [8]:
import numpy as np

def calc_conjugate(m, op_in, circuit):
    """
    Compute the effect of a Clifford circuit on an input Pauli operator under conjugation.
    Parameters:
    - m: Number of qubits.
    - op_in: Input Pauli operator as a tuple ('XYZ...', [qubit_indices]).
    - circuit: List of gates applied as tuples (Gate, [qubit_indices]).
    
    Returns:
    - op_out: Output Pauli operator in the same format as op_in.
    """
    # Define Pauli matrices and other gates
    I = np.eye(2)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)
    Y = 1j * X @ Z
    P = np.array([[1, 0], [0, 1j]], dtype=complex)
    H = (X + Z) / np.sqrt(2)

    e0 = np.array([[1], [0]], dtype=complex)
    e1 = np.array([[0], [1]], dtype=complex)
    E00 = e0 @ e0.T
    E11 = e1 @ e1.T
    CNOT = np.kron(E00, I) + np.kron(E11, X)
    CZ = np.kron(E00, I) + np.kron(E11, Z)

    # Initialize the input Pauli operators
    Pauli_in = [I] * m
    for i in range(m):
        if i + 1 in op_in[1]:
            Pli = op_in[0][op_in[1].index(i + 1)]
            Pauli_in[i] = {'X': X, 'Y': Y, 'Z': Z}.get(Pli, I)

    Pauli_out = Pauli_in[:]

    # Apply each gate in the circuit
    for gate, qubits in circuit:
        qubits = [q - 1 for q in qubits]  # Adjust to 0-indexing

        if gate == 'X':
            for q in qubits:
                Pauli_out[q] = X @ Pauli_out[q] @ X

        elif gate == 'Z':
            for q in qubits:
                Pauli_out[q] = Z @ Pauli_out[q] @ Z

        elif gate == 'Y':
            for q in qubits:
                Pauli_out[q] = Y @ Pauli_out[q] @ Y

        elif gate == 'P':
            for q in qubits:
                Pauli_out[q] = P @ Pauli_out[q] @ P

        elif gate == 'H':
            for q in qubits:
                Pauli_out[q] = H @ Pauli_out[q] @ H

        elif gate == 'CNOT':
            if len(qubits) != 2:
                raise ValueError("CNOT gate requires exactly two qubits.")
            control, target = qubits
            apply_cnot(Pauli_out, control, target, CNOT)

        elif gate == 'CZ':
            if len(qubits) != 2:
                raise ValueError("CZ gate requires exactly two qubits.")
            control, target = qubits
            apply_cz(Pauli_out, control, target, CZ)

        elif gate == 'Permute':
            desired_order = [q - 1 for q in qubits]
            if len(desired_order) != m:
                raise ValueError(f"Permutation requires {m} qubits.")
            Pauli_out = [Pauli_out[q] for q in desired_order]

    # Compute the final Pauli operator and sign
    op_out, out_sign = extract_pauli_operator(Pauli_out)

    # Adjust the sign
    if out_sign == -1:
        op_out = ('-' + op_out[0], [-1] + op_out[1])
    elif out_sign == 1j:
        op_out = ('i' + op_out[0], [1j] + op_out[1])
    elif out_sign == -1j:
        op_out = ('j' + op_out[0], [-1j] + op_out[1])

    return op_out

def apply_cnot(Pauli_out, control, target, CNOT):
    """Apply the CNOT gate."""
    out12 = find_other(CNOT, Pauli_out[control], 1)
    out21 = find_other(CNOT, Pauli_out[target], 2)
    Pauli_out[control] = out21 @ Pauli_out[control]
    Pauli_out[target] = Pauli_out[target] @ out12

def apply_cz(Pauli_out, control, target, CZ):
    """Apply the CZ gate."""
    out12 = find_other(CZ, Pauli_out[control], 1)
    out21 = find_other(CZ, Pauli_out[target], 2)
    Pauli_out[control] = out21 @ Pauli_out[control]
    Pauli_out[target] = Pauli_out[target] @ out12

def find_other(gate, inp, id):
    """Find the other output for a 2-qubit gate."""
    I = np.eye(2)
    Pin = np.kron(inp, I) if id == 1 else np.kron(I, inp)
    candidates = [np.kron(X, I), np.kron(Z, I), np.kron(Y, I), Pin]

    for candidate in candidates:
        if np.allclose(gate @ Pin @ gate.T.conj(), candidate):
            return candidate

    raise ValueError("Unknown gate encountered in find_other.")

def extract_pauli_operator(Pauli_out):
    """Extract the final Pauli operator and its sign."""
    op_out = ('', [])
    out_sign = 1

    for i, P in enumerate(Pauli_out):
        if np.allclose(P, X):
            op_out = (op_out[0] + 'X', op_out[1] + [i + 1])
        elif np.allclose(P, -X):
            op_out = (op_out[0] + 'X', op_out[1] + [i + 1])
            out_sign *= -1
        elif np.allclose(P, 1j * X):
            op_out = (op_out[0] + 'X', op_out[1] + [i + 1])
            out_sign *= 1j
        elif np.allclose(P, Z):
            op_out = (op_out[0] + 'Z', op_out[1] + [i + 1])
        elif np.allclose(P, Y):
            op_out = (op_out[0] + 'Y', op_out[1] + [i + 1])
        elif np.allclose(P, -Y):
            op_out = (op_out[0] + 'Y', op_out[1] + [i + 1])
            out_sign *= -1

    return op_out, out_sign

def mod2(matrix):
    """Perform element-wise mod 2 operation."""
    return np.mod(matrix, 2)

def find_symplectic(m, circuit):
    """Placeholder for the function that finds the symplectic matrix."""
    return symp_mat.find_symplectic(circuit)
def reduce_to_echelon_form(A):
    """
    Reduce the matrix A to echelon form over GF(2) and return the row operations matrix (M),
    column operations matrix (N), and the rank (rnk).
    """
    rows, cols = A.shape
    A = np.copy(A) % 2  # Ensure entries are in GF(2)

    # Initialize row and column operations as identity matrices
    M = np.eye(rows, dtype=int)  # Row operations matrix
    N = np.eye(cols, dtype=int)  # Column operations matrix

    row = 0  # Start from the top row

    for col in range(cols):
        if row >= rows:
            break  # Stop if all rows are processed

        # Find a pivot row for the current column
        pivot = np.where(A[row:, col] == 1)[0]
        if len(pivot) == 0:
            continue  # No pivot in this column, move to the next column

        pivot_row = pivot[0] + row  # Adjust pivot index to absolute position

        # Swap the current row with the pivot row
        A[[row, pivot_row]] = A[[pivot_row, row]]
        M[[row, pivot_row]] = M[[pivot_row, row]]  # Track row swap in M

        # Eliminate all other ones in this column
        for r in range(rows):
            if r != row and A[r, col] == 1:
                A[r] = (A[r] + A[row]) % 2  # Row elimination using GF(2) addition
                M[r] = (M[r] + M[row]) % 2  # Track the row operation in M

        # Track column swap in N if row != col (to ensure identity structure)
        if col != row:
            N[:, [col, row]] = N[:, [row, col]]  # Track column swap in N

        row += 1  # Move to the next row

    # Compute the rank as the number of non-zero rows in the reduced form
    rnk = np.sum(np.any(A, axis=1))

    return A, M, N, rnk


def gf2matinv(matrix):
    """
        Compute the inverse of a matrix in GF(2) using Gaussian elimination.

        Parameters:
        matrix (numpy.ndarray): A square matrix to invert in GF(2).

        Returns:
        numpy.ndarray: The inverse of the matrix in GF(2).

        Raises:
        ValueError: If the matrix is singular (i.e., not invertible).
    """
    matrix = np.array(matrix, dtype=np.int8) % 2  # Ensure GF(2) operations
    m = len(matrix)
    aug_matrix = np.concatenate((matrix, np.eye(m, dtype=np.int8)), axis=1)  # Augment with identity matrix

    # Gaussian elimination for GF(2)
    for col in range(m):
        for row in range(col, m):
            if aug_matrix[row, col]:
                if row != col:
                    aug_matrix[[col, row]] = aug_matrix[[row, col]]  # Swap rows
                break
        else:
            raise ValueError("Matrix is singular over GF(2)")

        for i in range(m):
            if i != col and aug_matrix[i, col]:
                aug_matrix[i] = (aug_matrix[i] + aug_matrix[col]) % 2  # Row operation

    return aug_matrix[:, m:]  # Return the inverse portion

def gf2lu(A):
    """
    Perform LU decomposition of A over GF(2) such that P @ A = L @ U,
    where P is a permutation matrix, L is a lower triangular matrix, 
    and U is an upper triangular matrix in row-echelon form.
    """
    m = A.shape[0]
    U = A.copy()
    L = np.eye(m, dtype=int)
    P = np.eye(m, dtype=int)

    for k in range(m - 1):
        # Find the pivot row
        pivot = np.where(U[k:m, k] == 1)[0]
        if len(pivot) == 0:
            continue
        i = pivot[0] + k  # Adjust to absolute row index

        # Swap rows in U, L, and P
        U[[k, i], k:] = U[[i, k], k:]
        L[[k, i], :k] = L[[i, k], :k]
        P[[k, i], :] = P[[i, k], :]

        # Perform row elimination
        for j in range(k + 1, m):
            L[j, k] = U[j, k]
            U[j, k:] = (U[j, k:] - L[j, k] * U[k, k:]) % 2

    return L, U, P

def symp_mat_decompose(F):
    """Decompose a symplectic matrix into elementary symplectic transformations."""
    m = F.shape[0] // 2
    I = np.eye(m, dtype=int)
    O = np.zeros((m, m), dtype=int)

    def U(k, m):
        """Creates a block diagonal matrix with an identity of size k 
        and a zero matrix of size (m-k)."""
        return np.block([
            [np.eye(k), np.zeros((k, m - k))],
            [np.zeros((m - k, k)), np.zeros((m - k, m - k))]
        ])
    def L(k, m):
        """Create a block diagonal matrix with zeros and I_k."""
        if k > m:
            raise ValueError("k should be less than or equal to m.")
        # Create the block matrix using np.block
        return np.block([
        [np.zeros((m - k, m - k), dtype=int), np.zeros((m - k, k), dtype=int)],
        [np.zeros((k, m - k), dtype=int), np.eye(k, dtype=int)]
        ])
    
    Omega = np.block([[O, I], [I, O]])  # Transversal Hadamard
    Elem1 = lambda Q: np.block([[Q, np.zeros_like(Q)], [np.zeros_like(Q), gf2matinv(Q)]])
    Elem2 = lambda R: np.block([[I, R], [O, I]])
    def Elem3(k, m):
        """Creates the Elem3 matrix using L and U."""
        upper = np.block([L(m - k, m), U(k, m)])
        lower = np.block([U(k, m), L(m - k, m)])
        return np.block([
            [upper],
            [lower]
        ])
    
    A = F[:m, :m]
    B = F[:m, m:]
    C = F[m:, :m]
    D = F[m:, m:]

    if ((np.all(A == I) and np.all(C == O) and np.all(D == I)) or
        (np.all(B == O) and np.all(C == O)) or
        np.all(F == Omega)):
        return [F]

    # Step 1
    _, M_A, N_A, k = reduce_to_echelon_form(A)
   
    Qleft1 = Elem1(M_A)
    Qright = Elem1(N_A)
    Fcp = mod2(Qleft1 @ F @ Qright)
    
    if k == m:
        Rright = Elem2(Fcp[:m, m:])
        Fcp = mod2(Fcp @ Rright)
        R = Fcp[m:, :m]
        return [gf2matinv(Qleft1), Omega, Elem2(R), Omega, gf2matinv(Rright), gf2matinv(Qright)]
    
    # Step 2
    Bmk = Fcp[k:m, m + k:]
    _, M_Bmk1, _, _ = reduce_to_echelon_form(Bmk)
    
    M_Bmk = np.block([[np.eye(k, dtype=int), np.zeros((k, m - k), dtype=int)],
                      [np.zeros((m - k, k), dtype=int), M_Bmk1]])
   
    Qleft2 = Elem1(M_Bmk)
   
    Fcp = mod2(Qleft2 @ Fcp)
   
    # Step 3
    E = Fcp[:k, m + k:]
    M_E = np.block([[np.eye(k, dtype=int), E], [np.zeros((m - k, k), dtype=int), np.eye(m - k, dtype=int)]])
    Qleft3 = Elem1(M_E)
    Fcp = mod2(Qleft3 @ Fcp)
    

    # Step 4
    S = Fcp[:k, :k]
    Rright = Elem2(np.block([[S, np.zeros((k, m - k), dtype=int)], [np.zeros((m - k, m), dtype=int)]]))
    Fcp = mod2(Fcp @ Rright)
    
    # Step 5
    
    
    Fright = mod2(Omega @ Elem3(k,m))
    Fcp = mod2(Fcp @ Fright)
    R = Fcp[m:, :m]

    Q = mod2(Qleft3 @ Qleft2 @ Qleft1)
    return [gf2matinv(Q), Omega, Elem2(R), Elem3(k,m), gf2matinv(Rright), gf2matinv(Qright)]

def find_circuit(F):
    """
    Find a quantum circuit for the given symplectic transformation F 
    using Trung Can's algorithm to decompose F into elementary gates.
    """
    m = F.shape[0] // 2
    I = np.eye(m, dtype=int)
    O = np.zeros((m, m), dtype=int)
    Omega = np.block([[O, I], [I, O]])

    # Validate symplectic matrix
    if not np.all(mod2(F @ Omega @ F.T) == Omega):
        print('\nInvalid symplectic matrix!')
        return []

    if np.all(F == np.eye(2 * m, dtype=int)):
        return []

    # Decompose F into elementary symplectic transformations
    Decomp = symp_mat_decompose(F)
    circuit = []

    for matrix in Decomp:
        if np.all(matrix == np.eye(2 * m, dtype=int)):
            continue
        elif np.all(matrix == Omega):
            circuit.append(('H', list(range(1, m + 1))))
            continue

        A = matrix[:m, :m]
        B = matrix[:m, m:]
        C = matrix[m:, :m]
        D = matrix[m:, m:]

        if np.all(A == I) and np.all(C == O) and np.all(D == I):
            # CZs and Phase gates
            P_ind = np.where(np.diag(B) == 1)[0].tolist()
            if P_ind:
                circuit.append(('P', P_ind))

            B = np.triu(mod2(B + np.diag(np.diag(B))))
            for j in range(m):
                CZ_ind = np.where(B[j] == 1)[0]
                for k in CZ_ind:
                    circuit.append(('CZ', [j + 1, k + 1]))

        elif np.all(B == O) and np.all(C == O):
            # CNOTs and Permutations using LU decomposition over GF(2)
            L, U, P = gf2lu(A)

            if not np.all(P == I):
                circuit.append(('Permute', (np.arange(1, m + 1) @ P.T).tolist()))

            for j in range(m):
                inds = np.setdiff1d(np.where(L[j] == 1)[0], [j])
                for k in inds:
                    circuit.append(('CNOT', [j + 1, k + 1]))

            for j in range(m - 1, -1, -1):
                inds = np.setdiff1d(np.where(U[j] == 1)[0], [j])
                for k in inds:
                    circuit.append(('CNOT', [j + 1, k + 1]))

        else:
            # Partial Hadamards
            k = m - np.sum(np.diag(A))
            Uk = np.block([[np.eye(k, dtype=int), np.zeros((k, m - k), dtype=int)]])
            Lmk = np.block([[np.zeros((m - k, k), dtype=int), np.eye(m - k, dtype=int)]])

            if np.all(A == Lmk) and np.all(B == Uk) and np.all(C == Uk) and np.all(D == Lmk):
                circuit.append(('H', list(range(1, k + 1))))
            else:
                print('\nUnknown elementary symplectic form!')
                return []

    return circuit

def gflineq(A, b):
    """Placeholder for solving linear equations in GF(2)."""
    return symplectic_ops.gf2_gaussian_elimination_with_echelon(A,b)

def qfind_all_symp_mat(U, H):
    """Placeholder for finding all symplectic matrices."""
    return symp_code.symplectic_code(U,H)

def get_conjugate_sign(j, m, circuit, pauli_type):
    """Calculate the sign under conjugation."""
    conj_ckt = calc_conjugate(m, [pauli_type, j], circuit)
    return -1 if conj_ckt[0][0] == '-' else 1

def convert_to_circuit(h, m):
    """Convert a complex vector to a circuit representation."""
    circuit = ['', []]
    for q in range(m):
        if h[q] == 1:
            circuit[0] += 'X'
            circuit[1].append(q + 1)
        elif h[q] == 1j:
            circuit[0] += 'Z'
            circuit[1].append(q + 1)
        elif h[q] == 1 + 1j:
            circuit[0] += 'Y'
            circuit[1].append(q + 1)
    return circuit

def report_error(j, m, h_ckt, h_new_ckt):
    """Report an error if the conjugation is incorrect."""
    if j <= m - k:
        print(f'\nSomething is wrong for logical Pauli X{j}!!')
    elif j > m:
        print(f'\nSomething is wrong for logical Pauli Z{j - m}!!')
    else:
        print(f'\nSomething is wrong for stabilizer {j - (m - k)}!!')

def get_cheapest_choice(choices, m):
    """Get the cheapest choice based on non-zero elements."""
    choices = choices[:, :m] + 1j * choices[:, m:]
    return choices[np.argmin(np.sum(choices != 0, axis=1))]

def append_single_qubit_gates(F_entry, x):
    """Append single-qubit gates to the circuit."""
    circuit = F_entry[1]
    if np.any(x.real == 1):
        circuit.append(('X', np.where(x.real == 1)[0] + 1))
    if np.any(x.imag == 1):
        circuit.append(('Z', np.where(x.imag == 1)[0] + 1))
    if np.any((x.real == 1) & (x.imag == 1)):
        circuit.append(('Y', np.where((x.real == 1) & (x.imag == 1))[0] + 1))
    F_entry[2] += 1  # Increase circuit depth




In [9]:
import numpy as np

def find_logical_cliff(S, Xbar, Zbar, circuit, Snorm=None, no_of_solns=1):
    """
    Find all symplectic matrices for a logical Clifford operator.
    
    Parameters:
    - S: Stabilizer matrix
    - Xbar, Zbar: Logical operators
    - circuit: Circuit description as a list of tuples (Gate, Qubits)
    - Snorm: Normalized stabilizer (default is S)
    - no_of_solns: Number of solutions to find (default is 1)
    
    Returns:
    - F_all: List of symplectic matrices and their corresponding circuits
    """
    if Snorm is None:
        Snorm = S  # Assume physical operator must centralize S

    k, n = S.shape
    m = n // 2
    tot = 2 ** (k * (k + 1) // 2)
    F_all = []

    # Step 1: Initial symplectic transformation
    Fin = find_symplectic(m - k, circuit)
    H = np.vstack([Xbar, Snorm, Zbar])
    H[:2 * (m - k)] = mod2(Fin @ H[:2 * (m - k)])

    # Step 2: Complete symplectic basis for F_2^(2m)
    U = np.vstack([Xbar, S, Zbar])
    for i in range(k):
        h = np.zeros(2 * m - k + i)
        h[m - k + i] = 1
        U[2 * m - k + i] = gflineq(np.fft.fftshift(U, axes=1), h)

    if no_of_solns == 1:
        F_all.append([find_symp_mat(U[:2 * m - k], H), [], 0])
    else:
        F_all = [[sol, [], 0] for sol in qfind_all_symp_mat(U, H)]

    # Step 3: Process each solution
    for i, F_entry in enumerate(F_all):
        F, _, _ = F_entry
        circuit = find_circuit(F)
        F_all[i][1] = circuit  # Store circuit
        F_all[i][2] = len(circuit)  # Circuit depth

        # Verify signs under conjugation
        v = np.zeros(2 * m - k, dtype=int)
        for j in range(2 * m - k):
            h = U[j, :m] + 1j * U[j, m:]
            h_new = H[j, :m] + 1j * H[j, m:]
            
            h_ckt = convert_to_circuit(h, m)
            h_new_ckt = convert_to_circuit(h_new, m)

            in_sign = get_conjugate_sign(j, m, circuit, 'X' if j <= m - k else 'Z')
            h_new_ckt, out_sign = calc_conjugate(m, h_ckt, circuit)

            if h_new_ckt == h_ckt and out_sign == -in_sign:
                v[j] = 1
            else:
                report_error(j, m, h_ckt, h_new_ckt)

        # Handle non-trivial signs
        if np.any(v == 1):
            choices = np.fft.fftshift(gflineq(H, v), axes=1)
            x = get_cheapest_choice(choices, m)
            append_single_qubit_gates(F_all[i], x)

    return F_all



In [10]:
import numpy as np

# Example input: 2 qubits (4x4 symplectic matrix)
# Stabilizer matrix S: [I I; 0 0]
S = np.array([
    [1, 0, 1, 0],
    [0, 1, 0, 1]
], dtype=int)

# Logical X-bar and Z-bar operators (each row corresponds to one qubit)
Xbar = np.array([
    [0, 1, 1, 0],
    [1, 0, 0, 1]
], dtype=int)

Zbar = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0]
], dtype=int)

# Clifford circuit as a list of tuples (Gate, [Qubit Indices])
circuit = [
    ('H', [1]),
    ('CNOT', [1, 2]),
    ('Z', [2]),
    ('CZ', [1, 2])
]

# Call the `find_logical_cliff` function with 2 solutions requested
F_all = find_logical_cliff(S, Xbar, Zbar, circuit, Snorm=None, no_of_solns=2)

# Print the results
print("Found Symplectic Matrices and Circuits:")
for i, (F, circ, depth) in enumerate(F_all):
    print(f"\nSolution {i + 1}:")
    print("Symplectic Matrix (F):")
    print(F)
    print("Circuit:")
    print(circ)
    print(f"Circuit Depth: {depth}")


ValueError: Matrix is singular over GF(2)